In [46]:
import numpy as np
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity

# pd.set_option('display.max_columns', None)     # Show all columns
import pandas as pd

pd.set_option('display.max_columns', None)     # Show all columns
pd.set_option('display.expand_frame_repr', False)  # Prevent wrapping
pd.set_option('display.width', 1000)           # Force wide display



In [47]:
books = pd.read_csv('E:/Book Reccomendation/archive/Books.csv')
users = pd.read_csv('E:/Book Reccomendation/archive/Users.csv')
ratings = pd.read_csv('E:/Book Reccomendation/archive/Ratings.csv')

books.head()

C:\Users\premp\AppData\Local\Temp\ipykernel_11784\357919912.py:1: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  books = pd.read_csv('E:/Book Reccomendation/archive/Books.csv')


,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...
2,0060973129,Decision in Normandy,Carlo D'Este,1991,HarperPerennial,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...
3,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999,Farrar Straus Giroux,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...
4,0393045218,The Mummies of Urumchi,E. J. W. Barber,1999,W. W. Norton &amp; Company,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...


In [48]:
books["Image-URL-S"]

0         http://images.amazon.com/images/P/0195153448.0...
1         http://images.amazon.com/images/P/0002005018.0...
2         http://images.amazon.com/images/P/0060973129.0...
3         http://images.amazon.com/images/P/0374157065.0...
4         http://images.amazon.com/images/P/0393045218.0...
                                ...                        
271355    http://images.amazon.com/images/P/0440400988.0...
271356    http://images.amazon.com/images/P/0525447644.0...
271357    http://images.amazon.com/images/P/006008667X.0...
271358    http://images.amazon.com/images/P/0192126040.0...
271359    http://images.amazon.com/images/P/0767409752.0...
Name: Image-URL-S, Length: 271360, dtype: object

In [49]:
print(users.head(),'\n\n')
print(ratings.head())


   User-ID                            Location   Age
0        1                  nyc, new york, usa   NaN
1        2           stockton, california, usa  18.0
2        3     moscow, yukon territory, russia   NaN
3        4           porto, v.n.gaia, portugal  17.0
4        5  farnborough, hants, united kingdom   NaN 


   User-ID        ISBN  Book-Rating
0   276725  034545104X            0
1   276726  0155061224            5
2   276727  0446520802            0
3   276729  052165615X            3
4   276729  0521795028            6


In [50]:
print(books.shape)
print(users.shape)
print(ratings.shape)  

(271360, 8)
(278858, 3)
(1149780, 3)


missing values


In [51]:
books.isnull().sum().sort_values(ascending=False)

Image-URL-L            3
Book-Author            2
Publisher              2
ISBN                   0
Year-Of-Publication    0
Book-Title             0
Image-URL-S            0
Image-URL-M            0
dtype: int64

In [52]:
books.value_counts('Publisher')

Publisher
Harlequin                                               7535
Silhouette                                              4220
Pocket                                                  3905
Ballantine Books                                        3783
Bantam Books                                            3646
                                                        ... 
school and library distribution by Silver Burdett Co       1
scholastic Inc.                                            1
scholastic Apple Paperbacks                                1
rossway Books                                              1
powerHouse Books                                           1
Name: count, Length: 16807, dtype: int64

In [53]:
users.isnull().sum()

User-ID          0
Location         0
Age         110762
dtype: int64

In [54]:
ratings.isnull().sum()  

User-ID        0
ISBN           0
Book-Rating    0
dtype: int64

Recommendations using popularity


In [55]:
rate_bname = ratings.merge(books,on='ISBN')
print(books.columns)
print(rate_bname.columns)
rate_bname.shape

Index(['ISBN', 'Book-Title', 'Book-Author', 'Year-Of-Publication', 'Publisher', 'Image-URL-S', 'Image-URL-M', 'Image-URL-L'], dtype='object')
Index(['User-ID', 'ISBN', 'Book-Rating', 'Book-Title', 'Book-Author', 'Year-Of-Publication', 'Publisher', 'Image-URL-S', 'Image-URL-M', 'Image-URL-L'], dtype='object')


(1031136, 10)

In [56]:
num_ratings = rate_bname.groupby('Book-Title').count()['Book-Rating'].reset_index() #groupby does not contain duplicates
num_ratings.rename(columns={'Book-Rating': 'Num-Ratings'},inplace=True)
print((num_ratings),'\n\n')


                                               Book-Title  Num-Ratings
0        A Light in the Storm: The Civil War Diary of ...            4
1                                   Always Have Popsicles            1
2                    Apple Magic (The Collector's series)            1
3        Ask Lily (Young Women of Faith: Lily Series, ...            1
4        Beyond IBM: Leadership Marketing and Finance ...            1
...                                                   ...          ...
241066                                      Ã?Â?lpiraten.            2
241067                     Ã?Â?rger mit Produkt X. Roman.            4
241068                                Ã?Â?sterlich leben.            1
241069                              Ã?Â?stlich der Berge.            3
241070                                  Ã?Â?thique en toc            2

[241071 rows x 2 columns] 




In [57]:
avg_rating = rate_bname.groupby('Book-Title')['Book-Rating'].mean().reset_index()
avg_rating.rename(columns={'Book-Rating':'avg_rating'},inplace=True)
avg_rating


,Book-Title,avg_rating
0,A Light in the Storm: The Civil War Diary of ...,2.250000
1,Always Have Popsicles,0.000000
2,Apple Magic (The Collector's series),0.000000
3,"Ask Lily (Young Women of Faith: Lily Series, ...",8.000000
4,Beyond IBM: Leadership Marketing and Finance ...,0.000000
...,...,...
241066,Ã?Â?lpiraten.,0.000000
241067,Ã?Â?rger mit Produkt X. Roman.,5.250000
241068,Ã?Â?sterlich leben.,7.000000
241069,Ã?Â?stlich der Berge.,2.666667


In [58]:
popular_books = num_ratings.merge(avg_rating,on='Book-Title')
popular_books

,Book-Title,Num-Ratings,avg_rating
0,A Light in the Storm: The Civil War Diary of ...,4,2.250000
1,Always Have Popsicles,1,0.000000
2,Apple Magic (The Collector's series),1,0.000000
3,"Ask Lily (Young Women of Faith: Lily Series, ...",1,8.000000
4,Beyond IBM: Leadership Marketing and Finance ...,1,0.000000
...,...,...,...
241066,Ã?Â?lpiraten.,2,0.000000
241067,Ã?Â?rger mit Produkt X. Roman.,4,5.250000
241068,Ã?Â?sterlich leben.,1,7.000000
241069,Ã?Â?stlich der Berge.,3,2.666667


In [59]:
# get top 50 popular books
popular_books = popular_books[popular_books['Num-Ratings'] >= 250].sort_values('avg_rating',ascending=False).head(50)
print(popular_books,'\n\n')
print(popular_books.shape,'\n\n')

                                               Book-Title  Num-Ratings  avg_rating
80434   Harry Potter and the Prisoner of Azkaban (Book 3)          428    5.852804
80422        Harry Potter and the Goblet of Fire (Book 4)          387    5.824289
80441      Harry Potter and the Sorcerer's Stone (Book 1)          278    5.737410
80426   Harry Potter and the Order of the Phoenix (Boo...          347    5.501441
80414    Harry Potter and the Chamber of Secrets (Book 2)          556    5.183453
191612  The Hobbit : The Enchanting Prelude to The Lor...          281    5.007117
187377  The Fellowship of the Ring (The Lord of the Ri...          368    4.948370
80445   Harry Potter and the Sorcerer's Stone (Harry P...          575    4.895652
211384     The Two Towers (The Lord of the Rings, Part 2)          260    4.880769
219741                              To Kill a Mockingbird          510    4.700000
183573                                  The Da Vinci Code          898    4.642539
1878

In [60]:
popular_books = books.merge(popular_books, on='Book-Title').drop_duplicates('Book-Title')

# Use the correct column names after merge (e.g., 'Book-Author_x', 'Image-URL-M_x')
popular_books = popular_books[['Book-Title', 'Book-Author_x', 'Image-URL-M_x', 'Num-Ratings', 'avg_rating']]
# Optionally, rename columns for clarity
popular_books.rename(columns={'Book-Author_x': 'Book-Author', 'Image-URL-M_x': 'Image-URL-M'}, inplace=True)
print(popular_books.head())

KeyError: "['Book-Author_x', 'Image-URL-M_x'] not in index"

# collabarative Filtering based Reccomendation System


In [ ]:
# users given maore than 200 ratings
x = rate_bname.groupby('User-ID').count()['Book-Rating'] >= 200
relevent_users = x[x].index
relevent_users


Index([   254,   2276,   2766,   2977,   3363,   4017,   4385,   6251,   6323,   6543,
       ...
       271705, 273979, 274004, 274061, 274301, 274308, 275970, 277427, 277639, 278418], dtype='int64', name='User-ID', length=816)

In [ ]:
filtered_rating = rate_bname[rate_bname['User-ID'].isin(relevent_users)]

print(filtered_rating)


         User-ID        ISBN  Book-Rating                                         Book-Title                    Book-Author Year-Of-Publication                             Publisher                                        Image-URL-S                                        Image-URL-M                                        Image-URL-L
1150      277427  002542730X           10  Politically Correct Bedtime Stories: Modern Ta...              James Finn Garner                1994             John Wiley &amp; Sons Inc  http://images.amazon.com/images/P/002542730X.0...  http://images.amazon.com/images/P/002542730X.0...  http://images.amazon.com/images/P/002542730X.0...
1151      277427  0026217457            0                 Vegetarian Times Complete Cookbook                     Lucy  Moll                1995                 John Wiley &amp; Sons  http://images.amazon.com/images/P/0026217457.0...  http://images.amazon.com/images/P/0026217457.0...  http://images.amazon.com/images/P/0026217457.

In [ ]:
y = filtered_rating.groupby('Book-Title').count()['Book-Rating'] > 50
famous_books = y[y].index
famous_books

Index(['1984', '1st to Die: A Novel', '2nd Chance', '4 Blondes', 'A Bend in the Road', 'A Case of Need', 'A Child Called \It\": One Child's Courage to Survive"', 'A Civil Action', 'A Day Late and a Dollar Short', 'A Fine Balance',
       ...
       'Winter Solstice', 'Wish You Well', 'Without Remorse', 'Wizard and Glass (The Dark Tower, Book 4)', 'Wuthering Heights', 'Year of Wonders', 'You Belong To Me', 'Zen and the Art of Motorcycle Maintenance: An Inquiry into Values', 'Zoya', '\O\" Is for Outlaw"'], dtype='object', name='Book-Title', length=683)

In [ ]:
final_ratings = filtered_rating[filtered_rating['Book-Title'].isin(famous_books)]
final_ratings

,User-ID,ISBN,Book-Rating,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L
1150,277427,002542730X,10,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc,http://images.amazon.com/images/P/002542730X.0...,http://images.amazon.com/images/P/002542730X.0...,http://images.amazon.com/images/P/002542730X.0...
1163,277427,0060930535,0,The Poisonwood Bible: A Novel,Barbara Kingsolver,1999,Perennial,http://images.amazon.com/images/P/0060930535.0...,http://images.amazon.com/images/P/0060930535.0...,http://images.amazon.com/images/P/0060930535.0...
1165,277427,0060934417,0,Bel Canto: A Novel,Ann Patchett,2002,Perennial,http://images.amazon.com/images/P/0060934417.0...,http://images.amazon.com/images/P/0060934417.0...,http://images.amazon.com/images/P/0060934417.0...
1168,277427,0061009059,9,One for the Money (Stephanie Plum Novels (Pape...,Janet Evanovich,1995,HarperTorch,http://images.amazon.com/images/P/0061009059.0...,http://images.amazon.com/images/P/0061009059.0...,http://images.amazon.com/images/P/0061009059.0...
1174,277427,006440188X,0,The Secret Garden,Frances Hodgson Burnett,1998,HarperTrophy,http://images.amazon.com/images/P/006440188X.0...,http://images.amazon.com/images/P/006440188X.0...,http://images.amazon.com/images/P/006440188X.0...
...,...,...,...,...,...,...,...,...,...,...
1029196,275970,1400031354,0,Tears of the Giraffe (No.1 Ladies Detective Ag...,Alexander McCall Smith,2002,Anchor,http://images.amazon.com/images/P/1400031354.0...,http://images.amazon.com/images/P/1400031354.0...,http://images.amazon.com/images/P/1400031354.0...
1029197,275970,1400031362,0,Morality for Beautiful Girls (No.1 Ladies Dete...,Alexander McCall Smith,2002,Anchor,http://images.amazon.com/images/P/1400031362.0...,http://images.amazon.com/images/P/1400031362.0...,http://images.amazon.com/images/P/1400031362.0...
1029270,275970,1573229725,0,Fingersmith,Sarah Waters,2002,Riverhead Books,http://images.amazon.com/images/P/1573229725.0...,http://images.amazon.com/images/P/1573229725.0...,http://images.amazon.com/images/P/1573229725.0...
1029309,275970,1586210661,9,Me Talk Pretty One Day,David Sedaris,2001,Time Warner Audio Major,http://images.amazon.com/images/P/1586210661.0...,http://images.amazon.com/images/P/1586210661.0...,http://images.amazon.com/images/P/1586210661.0...


to get books of author jnet`

In [ ]:
pt = final_ratings.pivot_table(index='Book-Title', columns='User-ID', values='Book-Rating')
print(pt)

User-ID                                             254     2276    2766    2977    3363    4017    4385    6251    6323    6543    6563    6575    7158    7286    7346    7915    8067    8245    8681    8936    10447   10819   11601     11676   11993   12538   12824   12982   13082   13273   13552   13850   14422   14521   15408   15418   16106   16634   16795   16916   16966   17950   18067   18401   19085   19664   20115   20201   21014   21252   21576   21659   22625   23288   23768   23872   23902   24194   24921   25409   25601   25981   26516   26535   26544   26583   26593   27617   28204   28360   28523   28591   29259   29855   30276   30487   30511   30533   30711   30735   30972   31315   31391   31556   31826   31846   32195   32440   32721   32773   33974   35050   35433   35836   35857   35859   36606   36609   36836   36907   37567   37712   37950   38023   38273   38781   39281   39467   39616   39646   39773   40889   40943   41084   41841   42914   43246   43806   43

In [61]:
pt.fillna(0, inplace=True)
print(pt)

User-ID                                             254     2276    2766    2977    3363    4017    4385    6251    6323    6543    6563    6575    7158    7286    7346    7915    8067    8245    8681    8936    10447   10819   11601     11676   11993   12538   12824   12982   13082   13273   13552   13850   14422   14521   15408   15418   16106   16634   16795   16916   16966   17950   18067   18401   19085   19664   20115   20201   21014   21252   21576   21659   22625   23288   23768   23872   23902   24194   24921   25409   25601   25981   26516   26535   26544   26583   26593   27617   28204   28360   28523   28591   29259   29855   30276   30487   30511   30533   30711   30735   30972   31315   31391   31556   31826   31846   32195   32440   32721   32773   33974   35050   35433   35836   35857   35859   36606   36609   36836   36907   37567   37712   37950   38023   38273   38781   39281   39467   39616   39646   39773   40889   40943   41084   41841   42914   43246   43806   43

In [62]:
from sklearn.metrics.pairwise import cosine_similarity

similarity_score = cosine_similarity(pt)
print(similarity_score.shape)  # Should be (number of books, number of books)

(683, 683)


In [65]:
def get_book_recommendations(book_name):
    if book_name not in pt.index:
        return "Book not found in the dataset."
    # Get the index of the book
    index = np.where(pt.index == book_name)[0][0]
    # Get the similarity scores for the book
    distance = similarity_score[index]
    
    # Get the indices of the books sorted by similarity score
    similar_item = sorted(list(enumerate(distance)), key=lambda x: x[1], reverse=True)[1:7]  # Exclude the book itself

    data =[]
    
    for i in similar_item:
        item = []
        temp_df = books[books['Book-Title'] == pt.index[i[0]]]
        item.append(temp_df.drop_duplicates("Book-Title")['Book-Title'].values[0])
        item.append(temp_df.drop_duplicates("Book-Title")['Book-Author'].values[0])
        item.append(temp_df.drop_duplicates("Book-Title")['Image-URL-M'].values[0])

        data.append(item)

        print(data)

    
    


get_book_recommendations('Animal Farm')  # Example book name

[['1984', 'George Orwell', 'http://images.amazon.com/images/P/0451524934.01.MZZZZZZZ.jpg']]
[['1984', 'George Orwell', 'http://images.amazon.com/images/P/0451524934.01.MZZZZZZZ.jpg'], ['Angus, Thongs and Full-Frontal Snogging: Confessions of Georgia Nicolson', 'Louise Rennison', 'http://images.amazon.com/images/P/0064472272.01.MZZZZZZZ.jpg']]
[['1984', 'George Orwell', 'http://images.amazon.com/images/P/0451524934.01.MZZZZZZZ.jpg'], ['Angus, Thongs and Full-Frontal Snogging: Confessions of Georgia Nicolson', 'Louise Rennison', 'http://images.amazon.com/images/P/0064472272.01.MZZZZZZZ.jpg'], ['Midnight', 'Dean R. Koontz', 'http://images.amazon.com/images/P/0425118703.01.MZZZZZZZ.jpg']]
[['1984', 'George Orwell', 'http://images.amazon.com/images/P/0451524934.01.MZZZZZZZ.jpg'], ['Angus, Thongs and Full-Frontal Snogging: Confessions of Georgia Nicolson', 'Louise Rennison', 'http://images.amazon.com/images/P/0064472272.01.MZZZZZZZ.jpg'], ['Midnight', 'Dean R. Koontz', 'http://images.amazon.

In [ ]:
import pickle
pickle.dump(popular_books,open('popular.pkl','wb'))

In [67]:
import pickle
pickle.dump(pt,open('pt.pkl','wb'))
pickle.dump(books,open('books.pkl','wb'))
pickle.dump(similarity_score,open('similarity_score.pkl','wb'))